## **Extract:**
*Extract* Amazon bestseller software and digital product data from the provided API.

In [1]:
import requests
import pandas as pd

In [2]:
api = 'https://data-liart.vercel.app/data'
response = requests.get(api)

data = response.json()

Reading the data into a dataframe:

In [3]:
df = pd.DataFrame(data['data'])
df.head()

,Unnamed: 0,rank,asin,product_title,product_price,product_star_rating,product_num_ratings,product_url,product_photo,rank_change_label,country,page
0,0,1,B073VKKNN9,Kaspersky | Premium - Total Security (Ultimate...,₹469.00,4.3,13324.0,https://www.amazon.in/dp/B073VKKNN9,https://images-eu.ssl-images-amazon.com/images...,None,IN,1
1,1,2,B07PQZJ6Y8,"K7 Security K7, Total Security, 1 User, 1 Year...",₹370.00,4.4,2291.0,https://www.amazon.in/dp/B07PQZJ6Y8,https://images-eu.ssl-images-amazon.com/images...,None,IN,1
2,2,3,B0D1KL34JM,Microsoft Office 2021 Professional - Lifetime ...,"₹1,799.00",4.5,388.0,https://www.amazon.in/dp/B0D1KL34JM,https://images-eu.ssl-images-amazon.com/images...,None,IN,1
3,3,4,B07B9YYLGG,"Bitdefender - 1 Device,1 Year - Mobile Securit...",₹94.00,4.1,9630.0,https://www.amazon.in/dp/B07B9YYLGG,https://images-eu.ssl-images-amazon.com/images...,None,IN,1
4,4,5,B073VLGMZ4,"McAfee Total Protection 2025 | 1 Device, 3 Yea...","₹1,699.00",4.4,5783.0,https://www.amazon.in/dp/B073VLGMZ4,https://images-eu.ssl-images-amazon.com/images...,None,IN,1


## **Transform:**

Creating a new dataframe from the required columns:

In [4]:
new_df = df[['product_title', 'product_price', 'product_star_rating', 'product_num_ratings', 'country']].copy()
new_df.head()

,product_title,product_price,product_star_rating,product_num_ratings,country
0,Kaspersky | Premium - Total Security (Ultimate...,₹469.00,4.3,13324.0,IN
1,"K7 Security K7, Total Security, 1 User, 1 Year...",₹370.00,4.4,2291.0,IN
2,Microsoft Office 2021 Professional - Lifetime ...,"₹1,799.00",4.5,388.0,IN
3,"Bitdefender - 1 Device,1 Year - Mobile Securit...",₹94.00,4.1,9630.0,IN
4,"McAfee Total Protection 2025 | 1 Device, 3 Yea...","₹1,699.00",4.4,5783.0,IN


#### **Derived Variables**

- *price_usd*  
Converts local currency price to USD
`product_price * exchange_rate`

In [5]:
new_df['country'].unique()

array(['IN', 'US', 'CA', 'AU', 'DE', 'FR', 'IT', 'ES', 'JP', 'MX'],
      dtype=object)

In [6]:
df['product_price'].astype(str).str.extract(r'([^\d.,\s]+)').value_counts() 

0   
€       392
$       356
₹       100
￥        97
None     54
Name: count, dtype: int64

In [7]:

# Extract currency symbol with regex
df['currency_symbol'] = df['product_price'].astype(str).str.extract(r'([^\d.,\s]+)')

# Drop nulls, then find unique symbols
unique_symbols = df['currency_symbol'].dropna().unique()

unique_symbols


array(['₹', '$', 'None', '€', '￥'], dtype=object)

Creating a new column `price_usd`:

In [8]:
new_df.head()

,product_title,product_price,product_star_rating,product_num_ratings,country
0,Kaspersky | Premium - Total Security (Ultimate...,₹469.00,4.3,13324.0,IN
1,"K7 Security K7, Total Security, 1 User, 1 Year...",₹370.00,4.4,2291.0,IN
2,Microsoft Office 2021 Professional - Lifetime ...,"₹1,799.00",4.5,388.0,IN
3,"Bitdefender - 1 Device,1 Year - Mobile Securit...",₹94.00,4.1,9630.0,IN
4,"McAfee Total Protection 2025 | 1 Device, 3 Yea...","₹1,699.00",4.4,5783.0,IN


In [9]:
# i. Define currency conversion rates 
currency_rates = {
    "₹": 0.012,   
    "$": 1.0,    
    "€": 1.08,    
    "£": 1.27    
}

# ii. Function to clean and convert price 
def convert_to_usd(price):
    if pd.isna(price):  
        return None
    
    # Extract currency symbol (first character)
    symbol = price.strip()[0]
    
    # Remove commas and symbol to get numeric value
    try:
        amount = float(price.replace(symbol, "").replace(",", "").strip())
    except ValueError:
        return None 
    
    # Look up rate in dictionary
    rate = currency_rates.get(symbol)
    if rate is None:    
        return None
    
    return amount * rate

# iii. Apply conversion
new_df["price_usd"] = new_df["product_price"].apply(convert_to_usd)

In [10]:
new_df.head()

,product_title,product_price,product_star_rating,product_num_ratings,country,price_usd
0,Kaspersky | Premium - Total Security (Ultimate...,₹469.00,4.3,13324.0,IN,5.628
1,"K7 Security K7, Total Security, 1 User, 1 Year...",₹370.00,4.4,2291.0,IN,4.440
2,Microsoft Office 2021 Professional - Lifetime ...,"₹1,799.00",4.5,388.0,IN,21.588
3,"Bitdefender - 1 Device,1 Year - Mobile Securit...",₹94.00,4.1,9630.0,IN,1.128
4,"McAfee Total Protection 2025 | 1 Device, 3 Yea...","₹1,699.00",4.4,5783.0,IN,20.388


In [11]:
#Display null rows of the product_price column
new_df[new_df['price_usd'].isnull()]

,product_title,product_price,product_star_rating,product_num_ratings,country,price_usd
158,"[Old Version] TurboTax Premier 2023, Federal &...",None,4.4,6483.0,US,NaN
163,Tech-Shop-pro install Key Included USB For Win...,None,4.1,159.0,US,NaN
183,[Old Version] TurboTax Business 2023 [PC Downl...,None,4.3,1014.0,US,NaN
267,TurboTax Standard 2023 - 8 Returns - English -...,None,4.0,235.0,CA,NaN
324,"3D Printer Tool Kit includes Debur Tool, Clean...",None,4.2,28.0,AU,NaN
...,...,...,...,...,...,...
976,Microsoft Office 2021 Professional Plus Editio...,None,4.0,49.0,MX,NaN
981,Ralix Windows Emergency Boot Disk - For Window...,None,3.3,2354.0,MX,NaN
989,Nuance Dragon USB Headset,None,4.0,718.0,MX,NaN
990,"TurboTax Deluxe 2023 Tax Software, Federal & S...",None,4.6,6407.0,MX,NaN


- *revenue_estimate*  
Estimates potential revenue per product
`product_price * product_num_ratings`

In [12]:
new_df['revenue_estimates'] = new_df['price_usd'] * new_df['product_num_ratings']
new_df.head()

,product_title,product_price,product_star_rating,product_num_ratings,country,price_usd,revenue_estimates
0,Kaspersky | Premium - Total Security (Ultimate...,₹469.00,4.3,13324.0,IN,5.628,74987.472
1,"K7 Security K7, Total Security, 1 User, 1 Year...",₹370.00,4.4,2291.0,IN,4.440,10172.040
2,Microsoft Office 2021 Professional - Lifetime ...,"₹1,799.00",4.5,388.0,IN,21.588,8376.144
3,"Bitdefender - 1 Device,1 Year - Mobile Securit...",₹94.00,4.1,9630.0,IN,1.128,10862.640
4,"McAfee Total Protection 2025 | 1 Device, 3 Yea...","₹1,699.00",4.4,5783.0,IN,20.388,117903.804


- *rating_bucket*  
Categorizes products based on star rating
`Low (1–2), Medium (2–3.5), High (3.5–4.5), Excellent (4.5–5)`

In [13]:
new_df['rating_bucket'] = pd.cut(new_df['product_star_rating'], bins=[1, 2, 3.5, 4.5, 5], labels=['Low', 'Medium', 'High', 'Excellent'], include_lowest=True, right=False).astype('object')
new_df['rating_bucket'] = new_df['rating_bucket'].fillna('Unknown')
new_df.head()

,product_title,product_price,product_star_rating,product_num_ratings,country,price_usd,revenue_estimates,rating_bucket
0,Kaspersky | Premium - Total Security (Ultimate...,₹469.00,4.3,13324.0,IN,5.628,74987.472,High
1,"K7 Security K7, Total Security, 1 User, 1 Year...",₹370.00,4.4,2291.0,IN,4.440,10172.040,High
2,Microsoft Office 2021 Professional - Lifetime ...,"₹1,799.00",4.5,388.0,IN,21.588,8376.144,Excellent
3,"Bitdefender - 1 Device,1 Year - Mobile Securit...",₹94.00,4.1,9630.0,IN,1.128,10862.640,High
4,"McAfee Total Protection 2025 | 1 Device, 3 Yea...","₹1,699.00",4.4,5783.0,IN,20.388,117903.804,High


- *review_density*  
Measures engagement relative to price
`product_num_ratings / product_price`

We'll need to:
- *Ensure the columns are numeric/float*
-  *Handle the null values in each of the columns*
- *Avoid division by zero*

In [14]:
new_df.dtypes

product_title           object
product_price           object
product_star_rating    float64
product_num_ratings    float64
country                 object
price_usd              float64
revenue_estimates      float64
rating_bucket           object
dtype: object

In [15]:
import numpy as np

# Replace NaNs with 0
new_df['price_usd'] = pd.to_numeric(new_df['price_usd'], errors='coerce').fillna(0.0)
new_df['product_num_ratings'] = pd.to_numeric(new_df['product_num_ratings'], errors='coerce').fillna(0.0)

def density(row):
    price = row['price_usd']
    ratings = row['product_num_ratings']
    
    if price == 0 or ratings == 0:
        return np.nan   
    return round(ratings / price, 4)

new_df['review_density'] = new_df.apply(density, axis=1).astype(float)

new_df.head()

,product_title,product_price,product_star_rating,product_num_ratings,country,price_usd,revenue_estimates,rating_bucket,review_density
0,Kaspersky | Premium - Total Security (Ultimate...,₹469.00,4.3,13324.0,IN,5.628,74987.472,High,2367.4485
1,"K7 Security K7, Total Security, 1 User, 1 Year...",₹370.00,4.4,2291.0,IN,4.440,10172.040,High,515.9910
2,Microsoft Office 2021 Professional - Lifetime ...,"₹1,799.00",4.5,388.0,IN,21.588,8376.144,Excellent,17.9729
3,"Bitdefender - 1 Device,1 Year - Mobile Securit...",₹94.00,4.1,9630.0,IN,1.128,10862.640,High,8537.2340
4,"McAfee Total Protection 2025 | 1 Device, 3 Yea...","₹1,699.00",4.4,5783.0,IN,20.388,117903.804,High,283.6472


- *category:*  
Classifies product into a software category
Extracted from `product_title` or metadata

In [16]:
new_df[['product_title']].head()

,product_title
0,Kaspersky | Premium - Total Security (Ultimate...
1,"K7 Security K7, Total Security, 1 User, 1 Year..."
2,Microsoft Office 2021 Professional - Lifetime ...
3,"Bitdefender - 1 Device,1 Year - Mobile Securit..."
4,"McAfee Total Protection 2025 | 1 Device, 3 Yea..."


In [ ]:
# Keyword mapping for classification
category_map = {
    "Kaspersky": "Security",
    "K7": "Security",
    "Bitdefender": "Security",
    "McAfee": "Security",
    "Norton": "Security",
    "Antivirus": "Security",
    "Office": "Productivity",
    "Word": "Productivity",
    "Excel": "Productivity",
    "PowerPoint": "Productivity",
    "Adobe": "Design",
    "Photoshop": "Design",
    "Illustrator": "Design",
    "Windows": "Operating System",
    "Linux": "Operating System",
    "Mac": "Operating System",
    "IDE": "Development",
    "Programming": "Development"
}

# Function to classify product
def classify_category(title):
    for keyword, category in category_map.items():
        if keyword.lower() in title.lower():
            return category
    return "Other"

# Apply to DataFrame
new_df["category"] = new_df["product_title"].apply(classify_category)

In [23]:
new_df.head()

,product_title,product_price,product_star_rating,product_num_ratings,country,price_usd,revenue_estimates,rating_bucket,review_density,category,brand
0,Kaspersky | Premium - Total Security (Ultimate...,₹469.00,4.3,13324.0,IN,5.628,74987.472,High,2367.4485,Security,Kaspersky
1,"K7 Security K7, Total Security, 1 User, 1 Year...",₹370.00,4.4,2291.0,IN,4.440,10172.040,High,515.9910,Security,K7
2,Microsoft Office 2021 Professional - Lifetime ...,"₹1,799.00",4.5,388.0,IN,21.588,8376.144,Excellent,17.9729,Productivity,Microsoft
3,"Bitdefender - 1 Device,1 Year - Mobile Securit...",₹94.00,4.1,9630.0,IN,1.128,10862.640,High,8537.2340,Security,Bitdefender
4,"McAfee Total Protection 2025 | 1 Device, 3 Yea...","₹1,699.00",4.4,5783.0,IN,20.388,117903.804,High,283.6472,Security,McAfee


- *brand*  
Extracts brand name from product title
String parsing or regex

In [ ]:
new_df["brand"] = new_df["product_title"].str.extract(r"^([A-Za-z0-9]+)")
new_df.head()

- *country_region*  
Groups countries into regions
E.g., North America, Europe, Asia, Oceania 

In [26]:
new_df['country'].unique()

array(['IN', 'US', 'CA', 'AU', 'DE', 'FR', 'IT', 'ES', 'JP', 'MX'],
      dtype=object)

In [ ]:
region_map = {
    # Asia
    "IN": "Asia",
    "JP": "Asia",

    # North America
    "US": "North America",
    "CA": "North America",
    "MX": "North America",

    # Oceania
    "AU": "Oceania",

    # Europe
    "DE": "Europe",
    "FR": "Europe",
    "IT": "Europe",
    "ES": "Europe"
}
new_df["country_region"] = new_df["country"].map(region_map)
new_df.head()

## **Load**

In [35]:
import sqlalchemy
from sqlalchemy import create_engine

In [ ]:
engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{db_name}?sslmode=require')

Loading transformed data into a postgres database:

In [39]:
new_df.to_sql('amazon_products', engine, if_exists='replace', index=False, schema='public')

999

Saving as a csv file:

In [38]:
new_df.to_csv('amazon_products.csv')